In [ ]:
# here we write out different versions of existing heatmap functions to implement droop volt-var + volt-watt control

In [ ]:
# changes to: my_heatmapSetup_funcs.py

In [ ]:
# changes to: my_detControlMatExistence_funcs.py

# indicMat is now 6n x 3n, [Fq Fp]'
def assignF(Fp,Fq,indicMat): # algo similar to updateStateSpace
    n=int(len(indicMat)/3) # indicMat is 6n x 3n
    
    # Make indicMat and F have the same sparsity structure, but different nonzero values
    Hblock1=np.zeros((3*n,3*n))   
    ridx,colidx=np.nonzero(indicMat[0:3*n,0:3*n]) # python indexing goes first to (last-1)          
    for k in range(len(ridx)):
            Hblock1[ridx[k]][colidx[k]] = Fq

    Hblock2=np.zeros((3*n,3*n))   
    ridx,colidx=np.nonzero(indicMat[3*n+1:,0:3*n]) 
    for k in range(len(ridx)):
            Hblock2[ridx[k]][colidx[k]] = Fp
  
    F=np.concatenate(Hblock1,Hblock2,axis=0)
    
    #print(F)
    print("Size of F=",F.shape)
    return F

def setupStateSpace(n, feeder, node_index_map, depths):
    #initializes state space matrices A and B
    #n = number of nodes in network
    #feeder = initiaized feeder object
    #node_index_map = dictionary of node indices with node names as keys
    A = np.identity(3*n)
    R, X = createRXmatrices_3ph(feeder, node_index_map,depths)
    B = np.concatenate((X, R), axis = 1)
    return A, B


# def updateStateSpace(feeder, n, act_locs, perf_nodes, node_index_map):
def updateStateSpace(feeder, n, voltvar_act,voltvar_perf,voltwatt_act,voltwatt_perf,node_index_map):
    #creates (6n*3n) matrix with 1 at (3i+ph)(3j+ph) for volt-watt control, and (3i+3n+ph)(3j+ph) for volt-var control
    #in the above description, ph is the integer representation (a=0, b=1, c=2) of the phase intersection between the actuator and performance nodes
    #if an actuator and performance node have no phases in common, a warning is printed
    #n = number of nodes in network
    #act_locs = list of actuators locations in network (list of strings)
    #perf_nodes = list of performance nodes 
    #node_index_map = dictionary of node indices for indicMat and F matrix
    indicMat = np.zeros((6*n,3*n))
    for i in range(len(act_locs)): 
        act = act_locs[i]
        perf = perf_nodes[i]
        act_phases = feeder.busdict[act[4:]].phases
        perf_phases = feeder.busdict[perf[4:]].phases
        act_index = node_index_map[act]
        perf_index = node_index_map[perf]
        
        phase_intrsct = [ph for ph in act_phases if ph in perf_phases]
        if phase_intrsct == []: # disallow configs in which the act and perf node phases are not aligned. Results in kgain=0.0001 and thinks it's feasible
            print('WARNING: act_node ' + act + ' can NOT track perf_node ' + perf + ' --> no common phases')
            phase_loop_check=False
            break # if any actuator of the config has phase mismtach (between act and perf), don't evaluate the config
        else:
            phase_loop_check=True
            for i in range(len(phase_intrsct)):
                if phase_intrsct[i] == 'a':
                    phase_intrsct[i] = 0
                elif phase_intrsct[i] == 'b':
                    phase_intrsct[i] = 1
                elif phase_intrsct[i] == 'c':
                    phase_intrsct[i] = 2

            for ph in phase_intrsct:
                indicMat[(act_index*3)+ph][(perf_index*3)+ph] = 1
                indicMat[(act_index*3)+(3*n)+ph][(perf_index*3)+(3*n)+ph] = 1   
            
    return indicMat,phase_loop_check